In [ ]:
import copy
import os.path
import sys

pwd = os.getcwd()
sys.path.append(os.path.dirname(pwd))

from utils.model_utils.evaluate import evaluate_model
from utils.model_utils.load_model import *
from utils.model_utils.model_config import ModelConfig
from utils.dataset_utils.load_dataset import load_data
from utils.decompose_utils.weight_remover import WeightRemoverBert
from utils.decompose_utils.concern_identification import ConcernIdentificationBert
from utils.decompose_utils.tangling_identification import TanglingIdentification
from transformers import AutoConfig
from utils.model_utils.save_module import save_module
from datetime import datetime
from utils.decompose_utils.concern_modularization import ConcernModularizationBert
from utils.decompose_utils.sampling import sampling_class
from utils.dataset_utils.load_dataset import convert_dataset_labels_to_binary, extract_and_convert_dataloader
import torch


In [ ]:
model_name = "sadickam/sdg-classification-bert"
model_type = "pretrained"
data = "OSDG"
num_labels = 16


# model_name = "textattack/bert-base-uncased-imdb"
# model_type = "pretrained"
# data = "IMDb"
# num_labels = 2

# model_name = "fabriceyhc/bert-base-uncased-yahoo_answers_topics"
# model_type = "pretrained"
# data = "Yahoo"
# num_labels = 10

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")

checkpoint_name = None
config = AutoConfig.from_pretrained(model_name, num_labels=num_labels)
model_config = ModelConfig(
    _model_name=model_name,
    _model_type=model_type,
    _data=data,
    _transformer_config=config,
    _checkpoint_name=checkpoint_name,
    _device=device,
)


In [ ]:
for i in range(num_labels):
    model, tokenizer, checkpoint = load_classification_model(model_config, train_mode=False)

    train_dataloader, valid_dataloader, test_dataloader = load_data(
        model_config, batch_size=32, test_size=0.3
    )

    print("Start Time:" + datetime.now().strftime("%H:%M:%S"))
    print("#Module " + str(i) + " in progress....")
    num_samples = 64

    positive_samples = sampling_class(
        train_dataloader, i, num_samples, num_labels, True, 4, device=device
    )
    negative_samples = sampling_class(
        train_dataloader, i, num_samples, num_labels, False, 4, device=device
    )

    all_samples = sampling_class(
        train_dataloader, 200, 20, num_labels, False, 4, device=device
    )

    print("origin")
    evaluate_model(model, model_config, test_dataloader)

    module = copy.deepcopy(model)
    wr = WeightRemoverBert(model, p=0.9)
    ci = ConcernIdentificationBert(model, p=0.4)
    ti = TanglingIdentification(model, p=0.5)

    print("Start Positive CI sparse")

    eval_step = 5
    for idx, batch in enumerate(all_samples):
        input_ids, attn_mask, _, total_sampled = batch
        with torch.no_grad():
            wr.propagate(module, input_ids)
        if idx % eval_step:
            # result = evaluate_model(module, model_config, test_dataloader)
            pass

    print("Start Positive CI after sparse")

    for idx, batch in enumerate(positive_samples):
        input_ids, attn_mask, _, total_sampled = batch
        with torch.no_grad():
            ci.propagate(module, input_ids)
        if idx % eval_step:
            # result = evaluate_model(module, model_config, test_dataloader)
            pass

    print("Start Negative TI")

    for idx, batch in enumerate(negative_samples):
        input_ids, attn_mask, _, total_sampled = batch
        with torch.no_grad():
            ti.propagate(module, input_ids)
        if idx % eval_step:
            # result = evaluate_model(module, model_config, test_dataloader)
            pass

    ConcernModularizationBert.channeling(module, ci.active_node, ti.dead_node, i, model_config.device)
    binary_module = ConcernModularizationBert.convert2binary(model_config, module)
    # save_module(binary_module, model_config.module_dir, model_config.model_name)
    
    for m in range(num_labels):
        converted_test_dataloader = convert_dataset_labels_to_binary(test_dataloader, i, True)
        result = evaluate_model(module, model_config, converted_test_dataloader)


Directory /home/Minwoo/LESN/Decompose/DecomposeBERT/Models/Configs/pretrained/sadickam/sdg-classification-bert exists.
Loading the model.
Start Time:18:13:36
#Module 0 in progress....
origin
Start Positive CI sparse
Start Positive CI after sparse
Start Negative TI
[1, 2, 7, 8, 9, 10, 15]
[0, 2, 7, 9, 10, 15]
[0, 9, 15, 0, 0, 0, 0, 12, 8, 16, 15, 0, 0, 0, 0, 16]
[16, 0, 10, 0, 0, 0, 0, 14, 0, 16, 16, 0, 0, 0, 0, 16]
0 1


Evaluating: 100%|██████████| 50/50 [00:24<00:00,  2.02it/s]


Loss: 0.3127
Precision: 0.8913, Recall: 0.8840, F1-Score: 0.8842
              precision    recall  f1-score   support

           0       0.94      0.82      0.88       775
           1       0.84      0.95      0.89       797

    accuracy                           0.88      1572
   macro avg       0.89      0.88      0.88      1572
weighted avg       0.89      0.88      0.88      1572

Directory /home/Minwoo/LESN/Decompose/DecomposeBERT/Models/Configs/pretrained/sadickam/sdg-classification-bert exists.
Loading the model.
Start Time:18:14:52
#Module 0 in progress....
origin
Start Positive CI sparse
Start Positive CI after sparse
Start Negative TI
[1, 2, 7, 8, 9, 10, 15]
[0, 2, 7, 9, 10, 15]
[0, 10, 9, 0, 0, 0, 0, 11, 8, 16, 15, 0, 0, 0, 0, 16]
[16, 0, 12, 0, 0, 0, 0, 9, 0, 16, 10, 0, 0, 0, 0, 16]
0 2


Evaluating: 100%|██████████| 50/50 [00:25<00:00,  1.92it/s]


Loss: 0.8319
Precision: 0.4752, Recall: 0.4950, F1-Score: 0.3690
              precision    recall  f1-score   support

           0       0.45      0.05      0.09       795
           1       0.50      0.94      0.65       797

    accuracy                           0.50      1592
   macro avg       0.48      0.50      0.37      1592
weighted avg       0.48      0.50      0.37      1592

Directory /home/Minwoo/LESN/Decompose/DecomposeBERT/Models/Configs/pretrained/sadickam/sdg-classification-bert exists.
Loading the model.
Start Time:18:16:02
#Module 0 in progress....
origin
Start Positive CI sparse
Start Positive CI after sparse
Start Negative TI
[2, 7, 9, 10, 12, 15]
[0, 7, 9, 10, 15]
[0, 0, 10, 0, 0, 0, 0, 9, 0, 16, 14, 0, 11, 0, 0, 16]
[16, 0, 0, 0, 0, 0, 0, 12, 0, 16, 14, 0, 0, 0, 0, 16]
0 3


...

Loss: 0.7271
Precision: 0.5310, Recall: 0.5136, F1-Score: 0.4578
              precision    recall  f1-score   support

           0       0.48      0.14      0.22       665
           1       0.58      0.89      0.70       882

    accuracy                           0.57      1547
   macro avg       0.53      0.51      0.46      1547
weighted avg       0.54      0.57      0.49      1547

Directory /home/Minwoo/LESN/Decompose/DecomposeBERT/Models/Configs/pretrained/sadickam/sdg-classification-bert exists.
Loading the model.
Start Time:20:18:53
#Module 5 in progress....
origin
Start Positive CI sparse
Start Positive CI after sparse
Start Negative TI
[0, 1, 2, 8, 9, 10, 12, 15]
[0, 2, 5, 9, 10, 12, 14, 15]
[16, 8, 9, 0, 0, 0, 0, 0, 8, 16, 14, 0, 10, 0, 0, 16]
[16, 0, 11, 0, 0, 16, 0, 0, 0, 16, 15, 0, 10, 0, 11, 16]
5 13


Evaluating: 100%|██████████| 38/38 [00:09<00:00,  3.98it/s]


Loss: 0.4938
Precision: 0.5160, Recall: 0.5110, F1-Score: 0.5048
              precision    recall  f1-score   support

           0       0.29      0.18      0.22       314
           1       0.74      0.85      0.79       882

    accuracy                           0.67      1196
   macro avg       0.52      0.51      0.50      1196
weighted avg       0.62      0.67      0.64      1196

Directory /home/Minwoo/LESN/Decompose/DecomposeBERT/Models/Configs/pretrained/sadickam/sdg-classification-bert exists.
Loading the model.
Start Time:20:20:23
#Module 5 in progress....
origin
Start Positive CI sparse
Start Positive CI after sparse
Start Negative TI
[0, 1, 2, 9, 10, 12, 15]
[0, 1, 2, 5, 9, 10, 12, 15]
[16, 10, 9, 0, 0, 0, 0, 0, 0, 16, 16, 0, 10, 0, 0, 16]
[16, 9, 9, 0, 0, 16, 0, 0, 0, 16, 16, 0, 13, 0, 0, 16]
5 14


Evaluating: 100%|██████████| 52/52 [00:17<00:00,  2.92it/s]


Loss: 0.5869
Precision: 0.4776, Recall: 0.4889, F1-Score: 0.4268
              precision    recall  f1-score   support

           0       0.42      0.13      0.20       756
           1       0.53      0.85      0.65       882

    accuracy                           0.52      1638
   macro avg       0.48      0.49      0.43      1638
weighted avg       0.48      0.52      0.44      1638

Directory /home/Minwoo/LESN/Decompose/DecomposeBERT/Models/Configs/pretrained/sadickam/sdg-classification-bert exists.
Loading the model.
